In [1]:
import pandas as pd
import numpy as np
import os

In [7]:
df = pd.read_csv('../final_stats/final_season_sent_game_last_1.csv', index_col=[0])
df = df.reset_index(drop=True)
df

,Week,Day,Date,home,away,home_score,away_score,away_positive96,away_positive24,away_neutral96,...,away_off_pass_att_last_1,away_off_pass_tds_last_1,away_off_pass_ints_last_1,away_def_sacks_last_1,away_def_sack_yds_last_1,away_tm_penalty_last_1,away_tm_penalty_yds_last_1,away_opp_penalty_last_1,away_opp_penalty_yds_last_1,away_spread
0,12,Sun,2022-11-27,titans,bengals,16.0,20.0,885.0,340.0,1098.0,...,39.0,4.0,2.0,2.0,16.0,6.0,44.0,8.0,70.0,-1.5
1,12,Sun,2022-11-27,browns,buccaneers,23.0,17.0,469.0,186.0,701.0,...,30.0,2.0,2.0,3.0,31.0,4.0,40.0,6.0,46.0,-3.0
2,12,Sun,2022-11-27,dolphins,texans,30.0,15.0,390.0,129.0,753.0,...,33.0,0.0,2.0,0.0,0.0,2.0,15.0,5.0,36.0,-14.0
3,12,Sun,2022-11-27,commanders,falcons,19.0,13.0,239.0,80.0,600.0,...,20.0,1.0,0.0,4.0,25.0,2.0,13.0,7.0,56.0,-4.0
4,12,Sun,2022-11-27,panthers,broncos,23.0,10.0,671.0,197.0,804.0,...,31.0,0.0,0.0,1.0,9.0,5.0,45.0,9.0,98.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,19,Sat,2023-01-14,jaguars,chargers,31.0,30.0,1845.0,973.0,1633.0,...,44.0,3.0,0.0,2.0,17.0,5.0,34.0,3.0,36.0,-2.5
102,19,Sun,2023-01-15,bills,dolphins,34.0,31.0,4213.0,2007.0,2233.0,...,31.0,0.0,0.0,0.0,0.0,5.0,47.0,6.0,54.0,-13.5
103,19,Sun,2023-01-15,vikings,giants,24.0,31.0,3100.0,1504.0,2097.0,...,40.0,1.0,0.0,3.0,22.0,8.0,66.0,4.0,30.0,-3.0
104,19,Sun,2023-01-15,bengals,ravens,24.0,17.0,283.0,86.0,793.0,...,44.0,0.0,2.0,2.0,13.0,5.0,22.0,3.0,35.0,-8.5


Drop unused columns
### No sentiment model

In [8]:
from no_sent_cols_to_remove import cols_to_remove

ModuleNotFoundError: No module named 'no_sent_cols_to_remove'